<a href="https://colab.research.google.com/github/The-Talking-Head/Source-code-and-Dataset/blob/master/Source%20Code/5_fold_Split_of_Grocery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **I have run this code on my pc in a local IDE, I have not run this code on google colab. This is given here for convenience**

In [ ]:
## The code for K-split

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import load_model

# Setting the path to dataset folders
base_folder = "D:/Grocery_master"

# Setting the desired image size for preprocessing
image_size = (224, 224)

# Loading the images and labels from the train, test, and validation folders using data generators
train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)
validation_datagen = ImageDataGenerator(rescale=1. / 255)

train_folder = os.path.join(base_folder, 'train')
train_generator = train_datagen.flow_from_directory(train_folder, target_size=image_size, batch_size=4, class_mode='categorical', color_mode='rgb')

test_folder = os.path.join(base_folder, 'test')
test_generator = test_datagen.flow_from_directory(test_folder, target_size=image_size, batch_size=4, class_mode='categorical', color_mode='rgb')


validation_folder = os.path.join(base_folder, 'validation')
validation_generator = validation_datagen.flow_from_directory(validation_folder, target_size=image_size, batch_size=4, class_mode='categorical', color_mode='rgb')

# Getting the number of classes
num_classes = train_generator.num_classes

# Preprocessing the labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_generator.classes)
train_labels = to_categorical(train_labels, num_classes=num_classes)

# Loading the base model from your device
base_model_path = "D:/GroceryKsplit/ensemble_model_vgg_mob"
base_model = load_model(base_model_path)

# Creating a new model and add the base model layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compiling the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Setting the number of folds for cross-validation
k = 5

# Performing K-fold cross-validation
kf = KFold(n_splits=k, shuffle=True)

# Initializing lists to store the evaluation metrics
accuracy_scores = []
loss_scores = []
models = []

# Iterating over the K folds
for fold, (train_index, val_index) in enumerate(kf.split(train_generator.filenames), 1):
    # Filtering the train and validation indices to avoid out-of-index problem
    train_index = train_index[train_index < len(train_generator)]
    val_index = val_index[val_index < len(train_generator)]

    # Getting the current fold data generators
    train_data_generator = train_datagen.flow_from_directory(
        train_folder,
        target_size=image_size,
        batch_size=4,
        class_mode='categorical',
        color_mode='rgb',
        subset='training',
        shuffle=True,
        seed=42
    )


    val_data_generator = validation_datagen.flow_from_directory(
        validation_folder,
        target_size=image_size,
        batch_size=4,
        class_mode='categorical',
        color_mode='rgb',
        shuffle=False
    )
    # Training the model on the current fold
    model.fit(train_data_generator, epochs=10, verbose=1)

    # Evaluating the model on the validation set of the current fold
    loss, accuracy = model.evaluate(val_data_generator)

    # Storing the evaluation metrics for the current fold
    accuracy_scores.append(accuracy)
    loss_scores.append(loss)

    # Saving the model for the current fold
    model.save(f"model_fold_{fold}.h5")
    models.append(f"model_fold_{fold}.h5")

# Calculating and printing the average evaluation metrics across all folds
avg_accuracy = np.mean(accuracy_scores)
avg_loss = np.mean(loss_scores)
print("Average Accuracy:", avg_accuracy)
print("Average Loss:", avg_loss)

# **Results found for the code**


D:\GroceryKsplit\venv\Scripts\python.exe D:\GroceryKsplit\main2.py
Found 2509 images belonging to 3 classes.
Found 356 images belonging to 3 classes.
Found 297 images belonging to 3 classes.
Found 2509 images belonging to 3 classes.
Found 297 images belonging to 3 classes.
Epoch 1/10
628/628 [==============================] - 707s 1s/step - loss: 0.3337 - accuracy: 0.8888
Epoch 2/10
628/628 [==============================] - 693s 1s/step - loss: 0.1908 - accuracy: 0.9195
Epoch 3/10
628/628 [==============================] - 694s 1s/step - loss: 0.1996 - accuracy: 0.9187
Epoch 4/10
628/628 [==============================] - 695s 1s/step - loss: 0.1584 - accuracy: 0.9426
Epoch 5/10
628/628 [==============================] - 692s 1s/step - loss: 0.1406 - accuracy: 0.9466
Epoch 6/10
628/628 [==============================] - 693s 1s/step - loss: 0.1395 - accuracy: 0.9446
Epoch 7/10
628/628 [==============================] - 690s 1s/step - loss: 0.1271 - accuracy: 0.9490
Epoch 8/10
628/628 [==============================] - 691s 1s/step - loss: 0.1054 - accuracy: 0.9578
Epoch 9/10
628/628 [==============================] - 696s 1s/step - loss: 0.1220 - accuracy: 0.9502
Epoch 10/10
628/628 [==============================] - 723s 1s/step - loss: 0.1169 - accuracy: 0.9562
75/75 [==============================] - 84s 1s/step - loss: 0.1227 - accuracy: 0.9394
Found 2509 images belonging to 3 classes.
Found 297 images belonging to 3 classes.
Epoch 1/10
628/628 [==============================] - 713s 1s/step - loss: 0.1103 - accuracy: 0.9617
Epoch 2/10
628/628 [==============================] - 712s 1s/step - loss: 0.0993 - accuracy: 0.9649
Epoch 3/10
628/628 [==============================] - 716s 1s/step - loss: 0.1035 - accuracy: 0.9625
Epoch 4/10
628/628 [==============================] - 704s 1s/step - loss: 0.1052 - accuracy: 0.9641
Epoch 5/10
628/628 [==============================] - 706s 1s/step - loss: 0.0950 - accuracy: 0.9701
Epoch 6/10
628/628 [==============================] - 695s 1s/step - loss: 0.1045 - accuracy: 0.9649
Epoch 7/10
628/628 [==============================] - 691s 1s/step - loss: 0.0885 - accuracy: 0.9697
Epoch 8/10
628/628 [==============================] - 691s 1s/step - loss: 0.0910 - accuracy: 0.9721
Epoch 9/10
628/628 [==============================] - 692s 1s/step - loss: 0.0804 - accuracy: 0.9765
Epoch 10/10
628/628 [==============================] - 692s 1s/step - loss: 0.0909 - accuracy: 0.9721
75/75 [==============================] - 82s 1s/step - loss: 0.1038 - accuracy: 0.9663
Found 2509 images belonging to 3 classes.
Found 297 images belonging to 3 classes.
Epoch 1/10
628/628 [==============================] - 690s 1s/step - loss: 0.0747 - accuracy: 0.9777
Epoch 2/10
628/628 [==============================] - 690s 1s/step - loss: 0.0743 - accuracy: 0.9753
Epoch 3/10
628/628 [==============================] - 691s 1s/step - loss: 0.0801 - accuracy: 0.9725
Epoch 4/10
628/628 [==============================] - 692s 1s/step - loss: 0.0814 - accuracy: 0.9761
Epoch 5/10
628/628 [==============================] - 692s 1s/step - loss: 0.0711 - accuracy: 0.9777
Epoch 6/10
628/628 [==============================] - 691s 1s/step - loss: 0.0748 - accuracy: 0.9805
Epoch 7/10
628/628 [==============================] - 694s 1s/step - loss: 0.0702 - accuracy: 0.9789
Epoch 8/10
628/628 [==============================] - 691s 1s/step - loss: 0.0630 - accuracy: 0.9793
Epoch 9/10
628/628 [==============================] - 692s 1s/step - loss: 0.0708 - accuracy: 0.9785
Epoch 10/10
628/628 [==============================] - 691s 1s/step - loss: 0.0687 - accuracy: 0.9781
75/75 [==============================] - 81s 1s/step - loss: 0.0849 - accuracy: 0.9663
Found 2509 images belonging to 3 classes.
Found 297 images belonging to 3 classes.
Epoch 1/10
628/628 [==============================] - 691s 1s/step - loss: 0.0561 - accuracy: 0.9849
Epoch 2/10
628/628 [==============================] - 692s 1s/step - loss: 0.0650 - accuracy: 0.9817
Epoch 3/10
628/628 [==============================] - 691s 1s/step - loss: 0.0606 - accuracy: 0.9821
Epoch 4/10
628/628 [==============================] - 692s 1s/step - loss: 0.0624 - accuracy: 0.9837
Epoch 5/10
628/628 [==============================] - 695s 1s/step - loss: 0.0595 - accuracy: 0.9833
Epoch 6/10
628/628 [==============================] - 699s 1s/step - loss: 0.0555 - accuracy: 0.9868
Epoch 7/10
628/628 [==============================] - 698s 1s/step - loss: 0.0588 - accuracy: 0.9809
Epoch 8/10
628/628 [==============================] - 690s 1s/step - loss: 0.0536 - accuracy: 0.9857
Epoch 9/10
628/628 [==============================] - 692s 1s/step - loss: 0.0470 - accuracy: 0.9864
Epoch 10/10
628/628 [==============================] - 691s 1s/step - loss: 0.0416 - accuracy: 0.9880
75/75 [==============================] - 82s 1s/step - loss: 0.1693 - accuracy: 0.9596
Found 2509 images belonging to 3 classes.
Found 297 images belonging to 3 classes.
Epoch 1/10
628/628 [==============================] - 691s 1s/step - loss: 0.0439 - accuracy: 0.9892
Epoch 2/10
628/628 [==============================] - 690s 1s/step - loss: 0.0355 - accuracy: 0.9916
Epoch 3/10
628/628 [==============================] - 690s 1s/step - loss: 0.0452 - accuracy: 0.9868
Epoch 4/10
628/628 [==============================] - 692s 1s/step - loss: 0.0392 - accuracy: 0.9912
Epoch 5/10
628/628 [==============================] - 692s 1s/step - loss: 0.0425 - accuracy: 0.9892
Epoch 6/10
628/628 [==============================] - 692s 1s/step - loss: 0.0489 - accuracy: 0.9841
Epoch 7/10
628/628 [==============================] - 690s 1s/step - loss: 0.0394 - accuracy: 0.9892
Epoch 8/10
628/628 [==============================] - 690s 1s/step - loss: 0.0432 - accuracy: 0.9900
Epoch 9/10
628/628 [==============================] - 692s 1s/step - loss: 0.0319 - accuracy: 0.9924
Epoch 10/10
628/628 [==============================] - 690s 1s/step - loss: 0.0411 - accuracy: 0.9904
75/75 [==============================] - 82s 1s/step - loss: 0.1376 - accuracy: 0.9731
Average Accuracy: 0.9609427690505982
Average Loss: 0.12364481687545777

Process finished with exit code 0

# **Importing the saved 5 models and calculating their accuracy, precision, recall and f1 score.**

In [ ]:
import os
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the path to your dataset folders
base_folder = "/content/drive/MyDrive/Grocery"
validation_folder = os.path.join(base_folder, 'validation')

# Set the desired image size for preprocessing
image_size = (224, 224)

# Load the images and labels from the validation folder using a data generator
validation_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = validation_datagen.flow_from_directory(validation_folder, target_size=image_size, batch_size=4, class_mode='categorical', color_mode='rgb', shuffle=False)

# Set the path to the folder containing the saved models
models_folder = "/content/drive/MyDrive/Grocery/Split result models"

# Initialize lists to store evaluation metrics
accuracies = []
precisions = []
recalls = []
f1_scores = []

# Load and evaluate the models
for i in range(1, 6):
    model_path = os.path.join(models_folder, f"model_fold_{i}.h5")
    model = load_model(model_path)

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(validation_generator, verbose=0)
    accuracies.append(accuracy)

    # Predict labels for the validation set
    y_pred = model.predict(validation_generator)
    y_true = validation_generator.classes

    # Convert predicted probabilities to class labels
    y_pred_labels = np.argmax(y_pred, axis=1)

    # Calculate precision, recall, and F1-score
    precision = precision_score(y_true, y_pred_labels, average='weighted')
    recall = recall_score(y_true, y_pred_labels, average='weighted')
    f1 = f1_score(y_true, y_pred_labels, average='weighted')

    # Append the metrics to the respective lists
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    # Print the metrics for the current fold
    print(f"Model {i}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print("")

# Calculate the average metrics across all folds
avg_accuracy = np.mean(accuracies)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1_score = np.mean(f1_scores)

# Print the average metrics
print("Average Metrics:")
print(f"Accuracy: {avg_accuracy}")
print(f"Precision: {avg_precision}")
print(f"Recall: {avg_recall}")
print(f"F1-score: {avg_f1_score}")


# **Results for the accuracy, prec, recall, f1 score**


Found 297 images belonging to 3 classes.
75/75 [==============================] - 177s 2s/step
Model 1:
Accuracy: 0.939393937587738
Precision: 0.9412497892921512
Recall: 0.9393939393939394
F1-score: 0.9376728378932705

75/75 [==============================] - 176s 2s/step
Model 2:
Accuracy: 0.9629629850387573
Precision: 0.9629111852451893
Recall: 0.9629629629629629
F1-score: 0.9628160654658908

75/75 [==============================] - 176s 2s/step
Model 3:
Accuracy: 0.9696969985961914
Precision: 0.969666403992645
Recall: 0.9696969696969697
F1-score: 0.9696352510278974

75/75 [==============================] - 180s 2s/step
Model 4:
Accuracy: 0.9629629850387573
Precision: 0.9637142399047162
Recall: 0.9629629629629629
F1-score: 0.9623499197898854

75/75 [==============================] - 179s 2s/step
Model 5:
Accuracy: 0.9764309525489807
Precision: 0.9766270285750804
Recall: 0.9764309764309764
F1-score: 0.9763404687852701

Average Metrics:
Accuracy: 0.962289571762085
Precision: 0.9628337294019564
Recall: 0.9622895622895623
F1-score: 0.9617629085924427